In [35]:
import pandas as pd
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

In [2]:
spotify_lyrics = pd.read_csv('./lyricspotify.csv')
missed = pd.read_csv('./lastfmmissed.csv')
lastfm = pd.read_csv('./lastfm.csv')

In [3]:
missed = missed[missed.columns[-8:]]

In [4]:
missed.head()

,album,artist,duration,listeners,playcount,single_release,track-title,track_no
0,Culture II,Migos,256,2964,8714,NaN,Higher We Go (Intro),1
1,Culture II,Migos,293,26420,91144,"30 May 2018, 04:02",Supastars,2
2,Culture II,Migos,255,55135,281136,NaN,Narcos,3
3,Culture II,Migos,251,25072,109135,NaN,BBO (Bad Bitches Only),4
4,Culture II,Migos,287,10354,32288,NaN,Auto Pilot,5


In [5]:
lastfm = lastfm[lastfm.columns[1:]]

In [6]:
last_fm_total = pd.concat([lastfm,missed])

In [7]:
last_fm_total = last_fm_total.rename(columns={'track-title':'song'})

In [8]:
last_fm_total['artist'] = last_fm_total['artist'].apply(lambda x: x.lower())
last_fm_total['artist'] = last_fm_total['artist'].apply(lambda x: x.replace(' ', '-'))

In [9]:
spotify_lyrics.drop_duplicates(subset=['song'], keep='first', inplace=True)

In [10]:
final = spotify_lyrics.merge(last_fm_total, on=['song', 'artist'])

In [11]:
final['playcount_percentage'] = final.groupby('album')['playcount'].apply(lambda x: x/(x.sum()))

In [13]:
final['is_hit'] = final['playcount_percentage'] >= .15

In [32]:
final = final[~final['danceability'].isna()]
final = final[~final['lyrics'].isna()]

In [45]:
def count_unique_words(lyrics):
    stop_words = set(stopwords.words('english')) 
    word_tokens = word_tokenize(lyrics) 
    no_stop_words_lyrics = [w for w in word_tokens if not w in stop_words] 
    unique = set(no_stop_words_lyrics)
    return len(unique)
    

In [46]:
for idx, row in final.iterrows():
 
  final.loc[idx, 'unique-words'] = count_unique_words(row['lyrics'])

In [44]:
count_unique_words(final['lyrics'][0])

In [47]:
final.head()

,Unnamed: 0,artist,song,lyrics,features,danceability,energy,loudness,speechiness,liveness,...,release_date,album,duration,listeners,playcount,single_release,track_no,playcount_percentage,is_hit,unique-words
0,19,drake,Survival,I been waitin' on this Yeah All of this disord...,NaN,0.607,0.596,-9.090,0.358,0.1230,...,2018-06-29,Scorpion,136,69548,229946,NaN,1,0.029865,False,159.0
1,20,drake,Nonstop,"Tay Keith, fuck these niggas up! Look, I just ...",NaN,0.912,0.412,-8.074,0.124,0.1040,...,2018-06-29,Scorpion,238,108576,750236,NaN,2,0.097441,False,224.0
2,21,drake,Elevate,"Ayy, elevate, elevate Only obligation is to te...",NaN,0.758,0.474,-8.720,0.146,0.1160,...,2018-06-29,Scorpion,185,63287,255611,NaN,3,0.033199,False,159.0
3,22,drake,Emotionless,"You've got me feeling emotions Ayy, higher Ahh...",NaN,0.413,0.677,-5.120,0.255,0.0793,...,2018-06-29,Scorpion,302,65061,273895,NaN,4,0.035574,False,226.0
6,25,drake,8 Out of 10,Too rich for who? Y'all just got rich again Wh...,NaN,0.529,0.690,-7.870,0.740,0.4980,...,2018-06-29,Scorpion,195,59637,210660,NaN,7,0.027361,False,201.0
